In [2]:
%matplotlib inline

import fockspy
from fockspy import fock_classes as fc


import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from scipy import sparse
from scipy.sparse import coo_matrix
from scipy.sparse import coo_array
from scipy.sparse import identity
from scipy.sparse import diags

### Fock spaces initialization 

In [3]:
m_c = 7
f = fc.fock_space(2*(1+m_c),14, statistics = 'Fermi')

### Problem parameters

In [7]:
K = f.dimension
m = f.modes

Eps_c = np.ones(m_c)
h = np.ones(m_c)
Eps_d = 1
V = np.ones(m_c)
U = 0.1

d_down = f.annihilate[0]
d_dag_down = f.create[0]
d_upp = f.annihilate[m_c+1]
d_dag_upp = f.create[m_c+1]

c_down = f.annihilate[1:m_c+1]
c_dag_down = f.create[1:m_c+1]
c_upp = f.annihilate[m_c+2:]
c_dag_upp = f.create[m_c+2:]


psi0 = np.zeros(K, dtype = complex)
j1 = 0
psi0[j1]=1


dt = 0.01
tmax = 100


### Hamiltonian building

#### $ H = \sum\limits_{i, \sigma } (\varepsilon_{i\sigma}  c^{+}_{i\sigma} c_{i\sigma} + h_{i} c^{+}_{i\sigma} c_{(i-1) \sigma} + h_{i} c^{+}_{i\sigma} c_{(i-1) \sigma}) + \sum\limits_{\sigma} \varepsilon_{d} d^{+}_{\sigma} d_{\sigma} + U d^{+}_{↑} d_{↑} d^{+}_{↓} d_{↓} + \sum\limits_{k, \sigma }  V_{k}(d^{+}_{\sigma}c_{k\sigma} + c^{+}_{k\sigma}d_{\sigma}) $

In [9]:
H = f.emptyH
for i in range(1,m_c):
    H += Eps_c[i]*c_dag_down[i]@c_down[i] + h[i]*c_dag_down[i]@c_down[i-1] + h[i]*c_down[i]@c_dag_down[i-1]
    H += Eps_c[i]*c_dag_upp[i]@c_upp[i] + h[i]*c_dag_upp[i]@c_upp[i-1] + h[i]*c_upp[i]@c_dag_upp[i-1]
H += Eps_d * d_dag_down@d_down + Eps_d * d_dag_upp@d_upp
H += U*d_dag_upp@d_upp@d_dag_down@d_down
for k in range(0,m_c):
    H += V[k]*(d_dag_down@c_down[k] + c_dag_down[k]@d_down)
    H += V[k]*(d_dag_upp@c_down[k] + c_dag_upp[k]@d_down)
    
    

In [15]:
H

<65519x65519 sparse matrix of type '<class 'numpy.complex128'>'
	with 917263 stored elements in Compressed Sparse Row format>